<a href="https://colab.research.google.com/github/gustavofurini/mestrado/blob/main/minssistema_especialista_inteligencia_artificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **API**

In [1]:
class RuleBase:
    def set_display(self, text):
        pass

    def trace(self, text):
        pass

    def display_variables(self, log):
        pass

    def display_rules(self, log):
        pass

    def reset(self):
        pass

    def backward_chain(self, goal_var_name):
        pass

    def forward_chain(self):
        pass

    def get_goal_variables(self):
        pass


In [2]:
class Variable:
    labels = None
    column = -1

    def __init__(self, name):
        self.name = name
        self.value = None

    def get_name(self):
        return self.name

    def set_value(self, value):
        self.value = value

    def get_value(self):
        return self.value

    def set_labels(self, new_labels):
        self.labels = []
        tok = new_labels.split(' ')
        for x in tok:
            self.labels.append(x)

    def get_label(self, index):
        return str(self.labels[index])

    def get_labels(self):
        return self.labels.copy()

    def get_labels_as_string(self):
        label_list = ''
        for x in self.labels:
            label_list += x + ' '
        return label_list

    def get_index(self, label):
        index = -1
        if self.labels is None:
            return index
        i = 0
        while i < len(self.labels):
            if label == self.labels[i]:
                index = i
                break
            i += 1
        return index

    def categorical(self):
        if self.labels is not None:
            return True
        else:
            return False

    def __str__(self):
        return self.name

    def to_string(self):
        return self.name

    def set_column(self, column):
        self.column = column

    def compute_statistics(self, in_value):
        pass

    def normalize(self, in_value, out_array, inx):
        pass

    @staticmethod
    def normalized_size():
        return 1

    @staticmethod
    def get_decoded_value(act, index):
        return str(act[index])


In [3]:

class RuleVariable(Variable):
    promptText = 'used to prompt user for value'
    ruleName = ''
    value = None

    def __init__(self, rb, name):
        super().__init__(name)
        self.rb = rb
        self.rb.add_variable(self)
        self.clauseRefs = []

    def set_value(self, value):
        self.value = value
        self.update_clauses()

    def ask_user(self):
        print("Ask User for Value")
        print(self.promptText)
        answer = input()
        self.set_value(answer)
        return self.value

    def add_clause_ref(self, ref):
        self.clauseRefs.append(ref)

    def update_clauses(self):
        for clause in self.clauseRefs:
            clause.check()

    def set_rule_name(self, rule_name):
        self.ruleName = rule_name

    def set_prompt_text(self, prompt_text):
        self.promptText = prompt_text

    def get_prompt_text(self):
        return self.promptText

    def compute_statistics(self, in_value):
        return in_value

    def normalize(self, in_value, out_array, inx):
        return inx


In [4]:
class Clause:
    def __init__(self, lhs, cond, rhs):
        self.lhs = lhs
        self.cond = cond
        self.rhs = rhs
        self.lhs.add_clause_ref(self)
        self.rule_refs = []
        self.truth = None
        self.consequent = False

    def __str__(self):
        return self.lhs.name + ' ' + self.cond.__str__() + ' ' + self.rhs + " "

    def add_rule_ref(self, ref):
        self.rule_refs.append(ref)

    def check(self):
        if self.consequent:
            self.truth = None
            return self.truth
        if self.lhs.value is None:
            self.truth = None
            return self.truth
        else:
            both_numeric = True
            lhs_numeric_value = None
            rhs_numeric_value = None
            try:
                lhs_numeric_value = float(self.lhs.value)
                rhs_numeric_value = float(self.rhs)
            except Exception as e:
                both_numeric = False
            if self.cond.index == 1:
                if both_numeric:
                    self.truth = lhs_numeric_value == rhs_numeric_value
                else:
                    self.truth = self.lhs.value.lower() == self.rhs.lower()
            elif self.cond.index == 2:
                if both_numeric:
                    self.truth = lhs_numeric_value > rhs_numeric_value
                else:
                    self.truth = self.lhs.value.lower() > self.rhs.lower()
            elif self.cond.index == 3:
                if both_numeric:
                    self.truth = lhs_numeric_value < rhs_numeric_value
                else:
                    self.truth = self.lhs.value.lower() < self.rhs.lower()
            elif self.cond.index == 4:
                if both_numeric:
                    self.truth = lhs_numeric_value != rhs_numeric_value
                else:
                    self.truth = self.lhs.value.lower() != self.rhs.lower()
            return self.truth

    def set_consequent(self):
        self.consequent = True

    def get_rule(self):
        if self.consequent:
            return self.rule_refs[0]
        else:
            return None


In [5]:
class Rule:
    fired = False

    def __init__(self, rb, name, lhs, rhs):
        if isinstance(lhs, Clause):
            self.rb = rb
            self.name = name
            self.antecedents = [None]
            self.antecedents[0] = lhs
            lhs.add_rule_ref(self)
            self.consequent = rhs
            rhs.add_rule_ref(self)
            rhs.set_consequent()
            rb.rule_list.append(self)
            self.truth = None
        else:
            self.__init__0(rb, name, lhs, rhs)

    def __init__0(self, rb, name, lhs_clauses, rhs):
        self.rb = rb
        self.name = name
        self.antecedents = []
        for x in range(len(lhs_clauses)):
            self.antecedents.append(None)

        i = 0
        while i < len(lhs_clauses):
            self.antecedents[i] = lhs_clauses[i]
            self.antecedents[i].add_rule_ref(self)
            i += 1
        self.consequent = rhs
        rhs.add_rule_ref(self)
        rhs.set_consequent()
        rb.rule_list.append(self)
        self.truth = None

    def num_antecedents(self):
        return len(self.antecedents)

    @staticmethod
    def check_rules(clause_refs):
        for clause in clause_refs:
            for rule in clause.rule_refs:
                rule.check()

    def check(self):
        self.rb.trace("\nTesting rule " + self.name)
        for i in range(len(self.antecedents)):
            if self.antecedents[i].truth is None:
                self.truth = None
                return self.truth
            if self.antecedents[i].truth:
                continue
            else:
                self.truth = False
                return self.truth
        self.truth = True
        return self.truth

    def fire(self):
        self.rb.trace("\nFiring rule " + self.name)
        self.truth = True
        self.fired = True
        if self.consequent.lhs is None:
            self.consequent.perform(self.rb)
        else:
            self.consequent.lhs.set_value(self.consequent.rhs)
            self.check_rules(self.consequent.lhs.clauseRefs)

    def back_chain(self):
        self.rb.trace("Evaluating rule " + self.name)
        print("\nEvaluating rule " + self.name)
        for i in range(len(self.antecedents)):
            if self.antecedents[i].truth is None:
                self.rb.backward_chain(self.antecedents[i].lhs.name)
            if self.antecedents[i].truth is None:
                self.antecedents[i].lhs.ask_user()
                self.truth = self.antecedents[i].check()
            if self.antecedents[i].truth:
                continue
            else:
                self.truth = False
                return self.truth
        self.truth = True
        return self.truth

    def display(self, log):
        log.append('\nRule-' + self.name + ":")
        i = 0
        first_time = True
        while i < len(self.antecedents):
            if first_time:
                aux = 'IF '
                first_time = False
            else:
                aux = ''
            next_clause = self.antecedents[i]
            if (i + 1) < len(self.antecedents):
                log.append(aux + ' ' + next_clause.__str__() + " AND")
            else:
                log.append(aux + ' ' + next_clause.__str__())
            i += 1
        log.append("THEN " + self.consequent.__str__())

    def reset(self):
        self.fired = False

In [6]:
class APP:
    def __init__(self, app):
        try:
            self.br = None
            self.ref_BR = None
            self.LOG = []
            self.LOG.append(app)
        except Exception as e:
            print("Exception lançada")
            print(e)

    def add_rule_base(self, ref_br):
        self.ref_BR = ref_br

    def show_log(self):
        for x in self.LOG:
            print(x, end=' ')

    def show_log_new_line(self):
        for x in self.LOG:
            print(x, end='\n')

    def clear_text(self):
        self.LOG.clear()
        self.LOG.append("")
        self.br.reset()

    def menu(self):
        naoCarregada = False
        while True:
            print('\n              : [0] sair', end='\n')
            print('        REGRAS: [1] carregar, [2] mostrar', end='\n')
            print(' FORWARD-CHAIN: [3] carregar variaveis, [4] executar, [7] reset', end='\n')
            print('BACKWARD-CHAIN: [5] carregar variaveis, [6] executar, [7] reset', end='\n')
            print('Digite sua opcao: ')
            option = int(input())

            if option == 0:
                return
            elif option == 1:
                if not naoCarregada:
                    self.br = self.ref_BR.create()
                    naoCarregada = True
                else:
                    print("Base de regas já foi carregada!!")
            elif option == 2:
                self.br.display_rules(self.LOG)
                self.show_log()
                self.LOG = []
            elif option == 3:
                self.ref_BR.demo_fc(self.LOG)
                self.show_log_new_line()
                self.LOG = []
            elif option == 4:
                self.LOG.append(" --- Starting Inferencing Cycle --- ")
                self.br.forward_chain()
                self.br.display_variables(self.LOG)
                self.show_log_new_line()
                self.LOG = []
            elif option == 5:
                self.ref_BR.demo_bc(self.LOG)
                self.show_log_new_line()
                self.LOG = []
            elif option == 6:
                goal_list = self.ref_BR.get_goal_list()
                print("Informe a variavel objetivo ", goal_list)
                goal = input()
                goal_var = self.br.get_variable(goal)
                if goal_var is not None:
                    self.br.backward_chain(goal)
                else:
                    self.LOG.append("goalVar.__name__: NAO ENCONTEADA")
                self.LOG.append(" --- Ending Inferencing Cycle --- ")
                if goal_var is not None:
                    result = goal_var.get_value()
                else:
                    result = "DESCONHECIDO"
                self.LOG.append("RESULTADO: " + str(result))
                self.br.display_variables(self.LOG)
                self.show_log_new_line()
                self.LOG = []
            elif option == 7:
                self.clear_text()


In [7]:
class Fact:
    fired = False

    def __init__(self, rb, name, f):
        self.rb = rb
        self.name = name
        self.fact = f
        self.rb.add_fact(self)
        self.truth = None

    def asserts(self, rb):
        if self.fired:
            return
        rb.trace("\nAsserting fact " + self.name)
        self.truth = True
        self.fired = True
        if self.fact.lhs is None:
            self.fact.perform(rb)
        else:
            self.fact.lhs.set_value(self.fact.rhs)

    def display(self, log):
        log.append(self.name + ": ")
        log.append(self.fact.__str__() + "\n")


In [8]:
class EffectorClause(Clause):
    truth = True
    consequent = True
    object = None

    def __init__(self, e_name, args):
        self.ruleRefs = []
        self.effectorName = e_name
        self.arguments = args

    def display(self):
        return "effector(" + self.effectorName + "," + self.arguments + ") "

    def perform(self, rb):
        self.object = rb.get_effector_object(self.effectorName)
        self.object.effector(self, self.effectorName, self.arguments)
        return self.truth


In [9]:
class Effector:
    def effector(self, obj, e_name, args):
        pass


In [10]:
class Condition:
    def __init__(self, symbol):
        self.symbol = symbol
        if symbol == "=":
            self.index = 1
        elif symbol == ">":
            self.index = 2
        elif symbol == "<":
            self.index = 3
        elif symbol == "!=":
            self.index = 4
        else:
            self.index = -1

    def __str__(self):
        return self.symbol

    def to_string(self):
        return self.symbol


In [11]:
class BooleanRuleBase(RuleBase):
    name = ''
    variable_list = {}
    clause_var_list = []
    rule_list = []
    conclusion_var_list = []
    rule_ptr = None
    clause_ptr = None
    goal_clause_stack = []
    effectors = {}
    sensors = {}
    factList = []
    log = []

    def set_display(self, text):
        self.log = text

    def __init__(self, name):
        self.name = name

    def get_name(self):
        return self.name

    def trace(self, text):
        if self.log is not None:
            self.log.append(text)

    def display_variables(self, text):
        for k, v in self.variable_list.items():
            text.append(v.name + " value = " + str(v.value))
        text.append("--------------------------------------")

    def display_rules(self, text):
        text.append("\n" + self.name + " Rule Base: " + "\n")
        for temp in self.rule_list:
            temp.display(text)
        if self.factList is not None:
            for temp in self.factList:
                temp.display(text)

    def display_conflict_set(self, rule_set):
        self.trace("\n" + " -- Rules in conflict set:\n")
        for temp in rule_set:
            self.trace(temp.name + "(" + str(temp.num_antecedents()) + "), ")

    def reset(self):
        self.trace(
            "\n --- Setting all " + str(self.name) + " variables to null")
        for k, v in self.variable_list.items():
            v.set_value(None)
        if self.factList is not None:
            for temp in self.factList:
                temp.fired = False
        for temp in self.rule_list:
            temp.reset()

    def backward_chain(self, goal_var_name):
        goal_var = self.variable_list[goal_var_name]
        for goal_clause in goal_var.clauseRefs:
            if not goal_clause.consequent:
                continue
            self.goal_clause_stack.append(goal_clause)
            goal_rule = goal_clause.get_rule()
            rule_truth = goal_rule.back_chain()

            if rule_truth is None:
                self.trace("\nRule " + goal_rule.name +
                           " is null, can't determine truth value.")
            elif rule_truth:
                goal_var.set_value(goal_clause.rhs)
                goal_var.set_rule_name(goal_rule.name)
                self.goal_clause_stack.pop()
                self.trace("\nRule " + goal_rule.name + " is true, setting " +
                           goal_var.name + ": = " + str(goal_var.value))
                if len(self.goal_clause_stack) == 0:
                    self.trace(
                        "\n +++ Found Solution for goal: " + goal_var.name)
                    break
            else:
                self.goal_clause_stack.pop()
                self.trace("\nRule " + goal_rule.name +
                           " is false, can't set " + goal_var.name)
        if goal_var.value is None:
            self.trace("\n +++ Could Not Find Solution for goal: " +
                       goal_var.name)

    def match(self, test):
        match_list = []
        for test_rule in self.rule_list:
            if test:
                test_rule.check()
            if test_rule.truth is None:
                continue
            if test_rule.truth and not test_rule.fired:
                match_list.append(test_rule)
        self.display_conflict_set(match_list)
        return match_list

    @staticmethod
    def select_rule(rule_set):
        max2 = None
        first_time_only = True
        best_rule = None
        for next_rule in rule_set:
            if first_time_only:
                first_time_only = False
                best_rule = next_rule
                max2 = best_rule.num_antecedents()
            num_clauses = next_rule.num_antecedents()
            if num_clauses > max2:
                max2 = num_clauses
                best_rule = next_rule
        return best_rule

    def forward_chain(self):
        conflict_rule_set = self.match(True)
        while len(conflict_rule_set) > 0:
            selected = self.select_rule(
                conflict_rule_set)  # select the "best" rule
            selected.fire()
            conflict_rule_set = self.match(False)

    def add_effector(self, obj, effector_name):
        if self.effectors is None:
            self.effectors = {}
        self.effectors[effector_name] = obj

    def get_effector_object(self, effector_name):
        return self.effectors[effector_name]

    def add_sensor(self, obj, sensor_name):
        if self.sensors is None:
            self.sensors = {}
        self.sensors[sensor_name] = obj

    def get_sensor_object(self, sensor_name):
        return self.sensors[sensor_name]

    def initialize_facts(self):
        if self.factList is not None:
            for fact in self.factList:
                fact.asserts(self)

    def add_fact(self, fact):
        if self.factList is None:
            self.factList = []
        self.factList.append(fact)

    def add_variable(self, variable):
        self.variable_list[variable.get_name()] = variable

    def get_variables(self):
        return self.variable_list.copy()

    def get_goal_variables(self):
        goal_vars = []
        for k, v in self.variable_list.items():
            goal_clauses = v.clauseRefs
            if (goal_clauses is not None) and (len(goal_clauses) != 0):
                goal_vars.append(v)
        return goal_vars

    def get_variable(self, name):
        if name in self.variable_list:
            return self.variable_list[name]
        return None

    def set_variable_value(self, name, value):
        variable = self.get_variable(name)
        if variable is not None:
            variable.set_value(value)
        else:
            print("BooleanRuleBase: Can't set value, variable "
                  + name + " is not defined!")


# **APP - NFL**

In [ ]:

class RuleBaseFootball:
    def __init__(self, nome, goals_list):
        self.br = BooleanRuleBase(nome)
        self.goals_list = goals_list

    def get_goal_list(self):
        return self.goals_list

    def create(self):
        distancia = RuleVariable(self.br, "distancia")
        distancia.set_labels("1 100")
        distancia.set_prompt_text("Qual é a distância até a end zone [1,100]?")

        tempo_restante = RuleVariable(self.br, "tempo_restante")
        tempo_restante.set_labels("0 60")
        tempo_restante.set_prompt_text("Quanto tempo resta no relógio [0,60]?")

        down_atual = RuleVariable(self.br, "down_atual")
        down_atual.set_labels("1 4")
        down_atual.set_prompt_text("Qual é o down atual [1,4]?")

        clima = RuleVariable(self.br, "clima")
        clima.set_labels("bom ruim")
        clima.set_prompt_text("Como está o clima [bom, ruim]?")

        tipo_de_jogada = RuleVariable(self.br, "tipo_de_jogada")
        tipo_de_jogada.set_labels("passe corrida field_goal punt")
        tipo_de_jogada.set_prompt_text("Qual jogada executar [passe, corrida, field_goal, punt]?")

        # Novas Variáveis
        posicao_campo = RuleVariable(self.br, "posicao_campo")
        posicao_campo.set_labels("defesa ataque")
        posicao_campo.set_prompt_text("Em qual parte do campo o time está [defesa, ataque]?")

        pontuacao_diferenca = RuleVariable(self.br, "pontuacao_diferenca")
        pontuacao_diferenca.set_labels("perdendo ganhando empate")
        pontuacao_diferenca.set_prompt_text("O time está ganhando, perdendo ou empatado?")

        forca_defesa = RuleVariable(self.br, "forca_defesa")
        forca_defesa.set_labels("forte fraca")
        forca_defesa.set_prompt_text("Qual é a força da defesa adversária [forte, fraca]?")

        pressao_defesa = RuleVariable(self.br, "pressao_defesa")
        pressao_defesa.set_labels("alta baixa")
        pressao_defesa.set_prompt_text("A defesa adversária está pressionando muito ou pouco [alta, baixa]?")

        estrategia_time = RuleVariable(self.br, "estrategia_time")
        estrategia_time.set_labels("agressiva conservadora")
        estrategia_time.set_prompt_text("Qual é a estratégia do time [agressiva, conservadora]?")

        c_equals = Condition("=")
        c_more_then = Condition(">")
        c_less_than = Condition("<")

        # Regras ajustadas
        Regras = [

            # Regra 01: Quando o time está longe da end zone (mais de 50 jardas),
            # é o primeiro ou segundo down, há mais de 30 segundos no relógio e o time está perdendo,
            # a melhor opção geralmente é um passe para tentar ganhar mais jardas rapidamente.
            Rule(self.br, "Regra 01",
                 [Clause(distancia, c_more_then, "50"),
                  Clause(down_atual, c_less_than, "2"),
                  Clause(tempo_restante, c_more_then, "30"),
                  Clause(pontuacao_diferenca, c_equals, "perdendo")],
                 Clause(tipo_de_jogada, c_equals, "passe")),

            # Regra 02: No quarto down, se o time está a menos de 40 jardas do gol e a defesa adversária é fraca,
            # vale a pena tentar um field goal, pois a chance de sucesso é razoável.
            Rule(self.br, "Regra 02",
                 [Clause(down_atual, c_equals, "4"),
                  Clause(distancia, c_less_than, "40"),
                  Clause(forca_defesa, c_equals, "fraca")],
                 Clause(tipo_de_jogada, c_equals, "field_goal")),

            # Regra 03: Se for o quarto down, a equipe está longe do gol (mais de 40 jardas)
            # e a defesa adversária está exercendo muita pressão, a melhor escolha é um punt para evitar um turnover.
            Rule(self.br, "Regra 03",
                 [Clause(down_atual, c_equals, "4"),
                  Clause(distancia, c_more_then, "40"),
                  Clause(pressao_defesa, c_equals, "alta")],
                 Clause(tipo_de_jogada, c_equals, "punt")),

            # Regra 04: Se o tempo está acabando (menos de 10 segundos), o time está próximo da end zone (menos de 20 jardas)
            # e a estratégia do time é agressiva, faz sentido tentar um passe para tentar marcar um touchdown rapidamente.
            Rule(self.br, "Regra 04",
                 [Clause(tempo_restante, c_less_than, "10"),
                  Clause(distancia, c_less_than, "20"),
                  Clause(estrategia_time, c_equals, "agressiva")],
                 Clause(tipo_de_jogada, c_equals, "passe")),

            # Regra 05: Se o clima está ruim e o time está perdendo, pode ser mais seguro correr com a bola,
            # pois passes são mais arriscados em condições adversas, como chuva ou vento forte.
            Rule(self.br, "Regra 05",
                 [Clause(clima, c_equals, "ruim"),
                  Clause(pontuacao_diferenca, c_equals, "perdendo")],
                 Clause(tipo_de_jogada, c_equals, "corrida")),

            # Regra 06: Se o time está dentro da linha de 5 jardas no campo de ataque,
            # faz sentido tentar uma jogada de corrida para chegar à end zone.
            Rule(self.br, "Regra 06",
                 [Clause(distancia, c_less_than, "5"),
                  Clause(posicao_campo, c_equals, "ataque")],
                 Clause(tipo_de_jogada, c_equals, "corrida")),

            # Regra 07: Se for o quarto down e o time precisa de apenas 2 jardas ou menos,
            # uma jogada de corrida pode ser a melhor opção para tentar converter o down.
            Rule(self.br, "Regra 07",
                 [Clause(down_atual, c_equals, "4"),
                  Clause(distancia, c_less_than, "2")],
                 Clause(tipo_de_jogada, c_equals, "corrida")),

            # Regra 08: Se for o quarto down e a distância até a end zone estiver entre 20 e 40 jardas,
            # a escolha mais lógica é tentar um field goal.
            Rule(self.br, "Regra 08",
                 [Clause(distancia, c_more_then, "20"),
                  Clause(distancia, c_less_than, "40"),
                  Clause(down_atual, c_equals, "4")],
                 Clause(tipo_de_jogada, c_equals, "field_goal")),

            # Regra 09: Se ainda há bastante tempo no relógio (mais de 20 segundos) e o clima está bom,
            # um passe é uma boa opção, pois há tempo suficiente para trabalhar a jogada.
            Rule(self.br, "Regra 09",
                 [Clause(tempo_restante, c_more_then, "20"),
                  Clause(clima, c_equals, "bom")],
                 Clause(tipo_de_jogada, c_equals, "passe")),

            # Regra 10: Se o tempo está acabando (menos de 10 segundos),
            # a única chance real de avançar rapidamente é um passe.
            Rule(self.br, "Regra 10",
                 [Clause(tempo_restante, c_less_than, "10")],
                 Clause(tipo_de_jogada, c_equals, "passe")),

            # Regra 11: Se for o terceiro down e faltar menos de 3 jardas para avançar,
            # uma jogada de corrida pode ser eficaz para garantir o avanço.
            Rule(self.br, "Regra 11",
                 [Clause(down_atual, c_equals, "3"),
                  Clause(distancia, c_less_than, "3")],
                 Clause(tipo_de_jogada, c_equals, "corrida")),

            # Regra 12: Se for o terceiro down e a distância para avançar estiver entre 3 e 10 jardas,
            # um passe pode ser a melhor opção para ganhar mais terreno.
            Rule(self.br, "Regra 12",
                 [Clause(down_atual, c_equals, "3"),
                  Clause(distancia, c_more_then, "3"),
                  Clause(distancia, c_less_than, "10")],
                 Clause(tipo_de_jogada, c_equals, "passe")),

            # Regra 13: Se o clima estiver ruim e ainda houver bastante tempo no relógio (mais de 30 segundos),
            # pode ser mais seguro correr com a bola, evitando erros causados por condições adversas.
            Rule(self.br, "Regra 13",
                 [Clause(clima, c_equals, "ruim"),
                  Clause(tempo_restante, c_more_then, "30")],
                 Clause(tipo_de_jogada, c_equals, "corrida")),

            # Regra 14: Se for o segundo down e a equipe precisa avançar mais de 10 jardas,
            # um passe é a jogada mais eficiente para ganhar terreno.
            Rule(self.br, "Regra 14",
                 [Clause(down_atual, c_equals, "2"),
                  Clause(distancia, c_more_then, "10")],
                 Clause(tipo_de_jogada, c_equals, "passe")),

            # Regra 15: Se for o primeiro down e o clima estiver ruim,
            # o time deve preferir uma jogada de corrida para minimizar o risco de turnovers.
            Rule(self.br, "Regra 15",
                 [Clause(down_atual, c_equals, "1"),
                  Clause(clima, c_equals, "ruim")],
                 Clause(tipo_de_jogada, c_equals, "corrida"))
        ]

        return self.br

    def demo_fc(self, LOG):
        LOG.append(" --- Ajustando valores para Futebol Americano (ForwardChain) ---")
        self.br.set_variable_value("distancia", "5")
        self.br.set_variable_value("tempo_restante", "9")
        self.br.set_variable_value("down_atual", "3")
        self.br.set_variable_value("clima", "ruim")
        self.br.set_variable_value("tipo_de_jogada", None)
        self.br.set_variable_value("posicao_campo", None)
        self.br.set_variable_value("pontuacao_diferenca", None)
        self.br.set_variable_value("forca_defesa", None)
        self.br.set_variable_value("pressao_defesa", None)
        self.br.set_variable_value("estrategia_time", None)
        self.br.display_variables(LOG)

    def demo_bc(self, LOG):
        LOG.append(" --- Ajustando valores para Futebol Americano (BackwardChain) ---")
        self.br.set_variable_value("distancia", None)
        self.br.set_variable_value("tempo_restante", None)
        self.br.set_variable_value("down_atual", None)
        self.br.set_variable_value("clima", None)
        self.br.set_variable_value("tipo_de_jogada", None)
        self.br.set_variable_value("posicao_campo", None)
        self.br.set_variable_value("pontuacao_diferenca", None)
        self.br.set_variable_value("forca_defesa", None)
        self.br.set_variable_value("pressao_defesa", None)
        self.br.set_variable_value("estrategia_time", None)
        self.br.display_variables(LOG)


class Main:
    def __init__(self):
        self.app = APP("Rule Application")

    def main(self):
        try:
            brFootball = RuleBaseFootball("Escolha da Jogada", "[tipo_de_jogada] :")
            self.app.add_rule_base(brFootball)
            self.app.menu()
        except Exception as e:
            print("Exception: RuleApp ", e.with_traceback())


if __name__ == '__main__':
    Main().main()
